# Miner Notebook : 01

This notebook is used for experimentation during the implementation of the Zimcoin miner component.

## Web References

- [OpenCL Scalar Data Types](https://registry.khronos.org/OpenCL/sdk/1.0/docs/man/xhtml/scalarDataTypes.html)

In [1]:
import sys
import logging
import numpy as np
import binascii
import pyopencl as cl
import miner
import hashlib
from miner import ZimcoinMiner
from Library.opencl_information import opencl_information
from blocks import Block
from transactions import Transaction

from importlib import reload

## Configure Logging

In [2]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
        logging.FileHandler("miner_notebook.log"),
        logging.StreamHandler(sys.stdout)
    ])

## Show the available OpenCL Platforms

In [3]:
info = opencl_information()
info.print_full_info()


OpenCL Platforms and Devices
Platform 0 - Name: NVIDIA CUDA
Platform 0 - Vendor: NVIDIA Corporation
Platform 0 - Version: OpenCL 3.0 CUDA 11.7.89
Platform 0 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: NVIDIA GeForce RTX 3060
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1867 Mhz
 Device - Compute Units: 28
 Device - Local Memory: 48 KB
 Device - Constant Memory: 64 KB
 Device - Global Memory: 12 GB
 Device - Max Buffer/Image Size: 3011 MB
 Device - Max Work Group Size: 1024


Platform 1 - Name: Portable Computing Language
Platform 1 - Vendor: The pocl project
Platform 1 - Version: OpenCL 1.2 pocl 1.6, None+Asserts, LLVM 9.0.1, RELOC, SLEEF, DISTRO, POCL_DEBUG
Platform 1 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: pthread-AMD Ryzen 5 3600 6-Core Processor
 Device - Type: ALL | CPU
 Device - Max Clock Speed: 4208 Mhz
 Device - Compute Units: 12
 Device - Local Memory: 512 K

## Create the Miner

In [19]:
reload(miner)

zimcoin_miner = ZimcoinMiner(0, 0)
print('Threads:', zimcoin_miner.thread_count)

Threads: 28672


## Calculate Hash Test

The input data does not fit neatly into 32 byte blocks so some padding and un-padding will be needed.

### Bit Shifting Tests

In [20]:
n32 = np.uint32(0)
n8 = np.uint8(0)

print('n32:', n32.nbytes)
print('n32:', n8.nbytes)

n32: 4
n32: 1


In [21]:
nonce = np.ulonglong(np.iinfo(np.ulonglong).max  // 2)
#nonce = np.ulonglong(5)

print('nonce:', nonce)
print('Nonce Bytes:', nonce.nbytes)
print('Nonce to Bytes :', nonce.tobytes())

print('64 bit integers :', np.frombuffer(nonce.tobytes(), dtype=np.uint64))
print('32 bit integers :', np.frombuffer(nonce.tobytes(), dtype=np.uint32))


int_nonce = int(nonce)
print('Integer Nonce:', int_nonce)
print(int_nonce & 0xFFFFFFFF)
print(int_nonce >> 32)

nonce: 9223372036854775807
Nonce Bytes: 8
Nonce to Bytes : b'\xff\xff\xff\xff\xff\xff\xff\x7f'
64 bit integers : [9223372036854775807]
32 bit integers : [4294967295 2147483647]
Integer Nonce: 9223372036854775807
4294967295
2147483647


In [22]:
seed = np.iinfo(np.uint64).max

target = np.uint64(seed * 0.7)
hash_1 = np.uint64(seed * 0.6)
hash_2 = np.uint64(seed * 0.8)
hash_3 = np.uint64(target - 10000)

print('Target       :', target)
print('Target Bytes :', target.nbytes)
print('Hash 1       :', hash_1)
print('Hash 2       :', hash_2)
print('Hash 3       :', hash_3)
print('Target 16-bit integers :', np.frombuffer(target.tobytes(), dtype=np.uint16))
print('Hash 1 16-bit (less)   :', np.frombuffer(hash_1.tobytes(), dtype=np.uint16))
print('Hash 2 16-bit (more)   :', np.frombuffer(hash_2.tobytes(), dtype=np.uint16))
print('Hash 3 16-bit (less)   :', np.frombuffer(hash_3.tobytes(), dtype=np.uint16))



Target       : 12912720851596685312
Target Bytes : 8
Hash 1       : 11068046444225730560
Hash 2       : 14757395258967642112
Hash 3       : 12912720851596675072
Target 16-bit integers : [12288 13107 13107 45875]
Hash 1 16-bit (less)   : [38912 39321 39321 39321]
Hash 2 16-bit (more)   : [53248 52428 52428 52428]
Hash 3 16-bit (less)   : [ 2048 13107 13107 45875]


### Helper Functions to create test blocks 

In [23]:
def get_test_first_block() -> Block:
    """
    Get a block representing the first in a chain for testing.
    """
    return Block(
            bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000000'),
            0,
            bytes.fromhex('4f3ea27a7af06cbe53911d4fb9326730d435255a'),
            [],
            1626626569,
            100000,
            bytes.fromhex('0000193f7397d8ed1a4991d91f8b8d2e55eb56915e884d435de7bbf0b183f335'),
            55419)

def get_transactions_block() -> Block:
    """
    Get a block with transactions for testing.
    """
    return Block(
            bytes.fromhex('0000193f7397d8ed1a4991d91f8b8d2e55eb56915e884d435de7bbf0b183f335'),
            1,
            bytes.fromhex('4f3ea27a7af06cbe53911d4fb9326730d435255a'),
            [
                Transaction(
                    bytes.fromhex("9e09208d54c012c0844cf17cfbb175157516dc90"),
                    bytes.fromhex("4f3ea27a7af06cbe53911d4fb9326730d435255a"),
                    bytes.fromhex("3056301006072a8648ce3d020106052b8104000a034200041a719dc420fdbdeef447e90a6368b9486d4afbacd900f6d9d5f62692dfa9ecb695999af4fcf61bdc523021b3aef2b84344b7c4ba7d3a36efe2e5f3eff50e8c54"),
                    390,
                    5,
                    1,
                    bytes.fromhex("3045022100fae9ab97090f2f0fb5715497e12a06438cbccc610bae2f9c019dfa5bdb40f0090220283f5498f22e17ac9ecf4c239d864811dd47cb0ccb8c3584794791fd171e6b90"),
                    bytes.fromhex("0cfd04ed0b2b279c12412687c770b1224c8bfed453292652694339ddade4d63a")),
            ],
            1626626571,
            100000,
            bytes.fromhex('000071f1c701e06e5b91adb4289d6c5227b614bd4441748923826e5d0e8828da'),
            83651)

### Test First Block Manual Hashing

In [24]:
# get the byte array to use for computing the hash
first_block = get_test_first_block()
first_block_bytes = first_block.to_bytes()

# create a numpy array from the bytes
first_block_data = np.frombuffer(first_block_bytes, dtype=np.uint32)
print('Input in 32-bit blocks:', first_block_data.size)
print(first_block_data)

Input in 32-bit blocks: 19
[         0          0          0          0          0          0
          0          0 2057453135 3194810490 1327337811  812069561
 1512388052 1626626569          0     100000          0          0
          0]


#### Test with uchar and no nonce

In [25]:
cl_ctx = zimcoin_miner.cl_context
cl_program = zimcoin_miner.build_program()
cl_queue = zimcoin_miner.cl_queue

# the plaintext to hash
first_block_bytes = first_block.to_bytes()
plaintext_bytes = np.frombuffer(first_block_bytes, dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))


# the hash output
hash_output = np.zeros(8, dtype=np.uint32)
leading_zeros = np.zeros(1, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_plaintext_bytes = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
    hostbuf=plaintext_bytes)

cl_plaintext_length = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
    hostbuf=plaintext_length)

cl_hash_output = cl.Buffer(
    cl_ctx,
    cl.mem_flags.WRITE_ONLY,
    hash_output.nbytes)

cl_leading_zeros = cl.Buffer(
    cl_ctx,
    cl.mem_flags.WRITE_ONLY,
    leading_zeros.nbytes)

# execute the program
cl_program.get_single_hash(
    cl_queue, (1,), None,
    cl_plaintext_bytes,
    cl_plaintext_length,
    cl_hash_output,
    cl_leading_zeros)

# get the results
cl.enqueue_copy(cl_queue, hash_output, cl_hash_output)
cl.enqueue_copy(cl_queue, leading_zeros, cl_leading_zeros)

# print the results
print("Leading Zeros: %d" % leading_zeros[0])
print("CL result      : %s" % binascii.hexlify(hash_output))
print("Correct result : %s" % binascii.hexlify(hashlib.sha256(first_block_bytes).digest()))

assert binascii.hexlify(hash_output) == binascii.hexlify(hashlib.sha256(first_block_bytes).digest())


Leading Zeros: 0
CL result      : b'1e76c28ddacdd9e94dfdcc155cc1359aa797762bc2a0c5eaec41e5a24bbb1300'
Correct result : b'1e76c28ddacdd9e94dfdcc155cc1359aa797762bc2a0c5eaec41e5a24bbb1300'


In [26]:
# get the byte array to use for computing the hash for the second block
second_block = get_transactions_block()
second_block_bytes = second_block.to_bytes()

# create a numpy array from the bytes
second_block_data = np.frombuffer(second_block_bytes, dtype=np.uint32)
print(second_block_data)

[1058603008 3990394739 3650177306  781028127 2438392661 1129154654
 4038846301  905151409 2057453135 3194810490 1327337811  812069561
 1512388052 3976527116 2619812619 2267431186  582054087 3573451596
 1378232659 3711517545  987161773 1626626571          0     100000
          0          0          0]


In [27]:
zimcoin_miner.build_program()

block_bytes = second_block.to_bytes()
block_data = np.frombuffer(block_bytes, dtype=np.uint32)

# set the other variables to send to the kernel
nonce = np.zeros(shape=1, dtype=np.ulonglong)
input_len = np.int32(block_data.size) * 4 # length in bytes
print(input_len)
hash = np.zeros(8, dtype=np.uint32)

# allocate the memory for the variables on the device
cl_block_data = cl.Buffer(zimcoin_miner.cl_context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=block_data)
cl_input_len = cl.Buffer(zimcoin_miner.cl_context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=input_len)
cl_nonce = cl.Buffer(zimcoin_miner.cl_context, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_hash = cl.Buffer(zimcoin_miner.cl_context, cl.mem_flags.WRITE_ONLY, hash.nbytes)

zimcoin_miner.cl_program.get_hashed_nonce(
    zimcoin_miner.cl_queue, (zimcoin_miner.cl_threads,), None,
    cl_block_data,
    cl_input_len,
    cl_nonce,
    cl_hash)

# get the results
cl.enqueue_copy(zimcoin_miner.cl_queue, nonce, cl_nonce)
cl.enqueue_copy(zimcoin_miner.cl_queue, hash, cl_hash)


# show the results
calculated_hash = hashlib.sha256()
calculated_hash.update(block_bytes)
print(nonce[0].tobytes())
calculated_hash.update(nonce[0].tobytes())    
#calculated_hash.update(int(nonce[0]).to_bytes(8, byteorder='little', signed=False))    

output_hash = hash.tobytes()

print('Output Nonce:', nonce[0])
print('Output Hash     :', binascii.hexlify(output_hash))
print('Calculated Hash :', binascii.hexlify(calculated_hash.digest()))
print()

print('-- Now do it the other way around')
little_nonce = int.from_bytes(nonce[0].tobytes(), byteorder='little')
little_bytes = little_nonce.to_bytes(8, byteorder='little', signed=False)
little_hash = hashlib.sha256()
little_hash.update(block_bytes)
little_hash.update(little_bytes)

print('So the nonce is:', little_nonce)
print('And the hash is:', binascii.hexlify(little_hash.digest()))


108
b'\xff\xff\xff\xff\xff\xff\xff\x7f'
Output Nonce: 9223372036854775807
Output Hash     : b'0302cdab8a819ed8ee0216223b4ab026614bde91b207ae27d05a6717d4c179d0'
Calculated Hash : b'0302cdab8a819ed8ee0216223b4ab026614bde91b207ae27d05a6717d4c179d0'

-- Now do it the other way around
So the nonce is: 9223372036854775807
And the hash is: b'0302cdab8a819ed8ee0216223b4ab026614bde91b207ae27d05a6717d4c179d0'


## Comparing against the target

In [28]:
from attr import has


block = get_transactions_block()
target = np.frombuffer(
    (2 ** 256 // block.difficulty).to_bytes(32, byteorder='big', signed=False),
    np.uint32)

#hash_bytes = binascii.unhexlify('db2b6dcdeea9a1074ca5157844a853d7993a516a7108b253a3a89418b8043443')
#hash_bytes = binascii.unhexlify('000000ffffffffffffffffffffffffffffffffffffffffffffffffffffffffff')
hash_bytes = binascii.unhexlify('f9c27560fc94a697da11d177d8c949fca1d2f3a13d8ea234518e95b98bd45a6f')

hash = np.frombuffer(
    hash_bytes,
    np.uint32)

print('Target (hex) :', binascii.hexlify(target))
print('Hash (hex)   :', binascii.hexlify(hash))
print()
print('Target       :', target)
print('Hash         :', hash)

for i in range(0, 8):
    print(f'{hash[i] < target[i]}', end=', ')

print('\n')
for i in range(0, 8):
    print(binascii.hexlify(hash[i]), binascii.hexlify(target[i]))

print('\n')
for i in range(0, 8):
    print(np.frombuffer(hash[i], np.uint8), ' - ', np.frombuffer(target[i], np.uint8))

Target (hex) : b'0000a7c5ac471b4784230fcf80dc33721d53cddd6e04c059210385c67dfe32a0'
Hash (hex)   : b'f9c27560fc94a697da11d177d8c949fca1d2f3a13d8ea234518e95b98bd45a6f'

Target       : [3316056064 1192970156 3473875844 1916001408 3721220893 1505756270
 3330605857 2687696509]
Hash         : [1618330361 2544276732 2010190298 4232694232 2717110945  883068477
 3113586257 1868223627]
True, False, True, False, True, True, True, True, 

b'f9c27560' b'0000a7c5'
b'fc94a697' b'ac471b47'
b'da11d177' b'84230fcf'
b'd8c949fc' b'80dc3372'
b'a1d2f3a1' b'1d53cddd'
b'3d8ea234' b'6e04c059'
b'518e95b9' b'210385c6'
b'8bd45a6f' b'7dfe32a0'


[249 194 117  96]  -  [  0   0 167 197]
[252 148 166 151]  -  [172  71  27  71]
[218  17 209 119]  -  [132  35  15 207]
[216 201  73 252]  -  [128 220  51 114]
[161 210 243 161]  -  [ 29  83 205 221]
[ 61 142 162  52]  -  [110   4 192  89]
[ 81 142 149 185]  -  [ 33   3 133 198]
[139 212  90 111]  -  [125 254  50 160]


## Mine Block

In [29]:
reload(miner)

zimcoin_miner = ZimcoinMiner(0, 0)
print('Threads:', zimcoin_miner.thread_count)

Threads: 28672


### Test Kernel Function

In [30]:
zimcoin_miner = ZimcoinMiner(0, 0)
print('Threads:', zimcoin_miner.thread_count)

cl_ctx = zimcoin_miner.cl_context
cl_program = zimcoin_miner.build_program()
cl_queue = zimcoin_miner.cl_queue

# set the variables to test the kernel function
block = get_transactions_block()
seed = np.ulonglong(500_000_000)
window_size = np.uint32(10000)
block_data = np.frombuffer(block.to_bytes(), dtype=np.uint32)
block_data_len = np.int32(block_data.size) * 4 # length in bytes
nonce = np.zeros(shape=1, dtype=np.ulonglong)

# target = np.frombuffer(
#     0x0000ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff \
#     .to_bytes(32, byteorder='big', signed=False),
#     np.uint32)

# NOTE: the difficulty is multiplied by 1000 for testing purposes
target = np.frombuffer(
    (2 ** 256 // (block.difficulty * 1000) ).to_bytes(32, byteorder='big', signed=False),
    np.uint32)

hash = np.zeros(8, dtype=np.uint32)

print('Target (hex) :', binascii.hexlify(target))
print('Target       :', target)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_window_size = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=window_size)
cl_block_data = cl.Buffer(zimcoin_miner.cl_context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=block_data)
cl_block_data_len = cl.Buffer(zimcoin_miner.cl_context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=block_data_len)
cl_nonce = cl.Buffer(zimcoin_miner.cl_context, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_target = cl.Buffer(zimcoin_miner.cl_context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=target)
cl_hash = cl.Buffer(zimcoin_miner.cl_context, cl.mem_flags.WRITE_ONLY, hash.nbytes)

# execute the kernel
cl_program.mine_sequential(
    cl_queue, (zimcoin_miner.thread_count,), None,
    cl_seed,
    cl_window_size,
    cl_block_data,
    cl_block_data_len,
    cl_nonce,
    cl_target,
    cl_hash)

# get the results
cl.enqueue_copy(zimcoin_miner.cl_queue, nonce, cl_nonce)
cl.enqueue_copy(zimcoin_miner.cl_queue, hash, cl_hash)


# show the results
print('Out Hash     :', hash)

final_nonce = int.from_bytes(nonce[0].tobytes(), byteorder='little')

calculated_hash = hashlib.sha256()
calculated_hash.update(block.to_bytes())
calculated_hash.update(final_nonce.to_bytes(8, byteorder='little', signed=False))    

print('Nonce   :', final_nonce)
print('Hash    :', binascii.hexlify(calculated_hash.digest()))
print('Out Hash:', binascii.hexlify(hash))

Threads: 28672
Target (hex) : b'0000002af31dc4611873bf3f70834acdae9f0f4f534f5d60585a5f1c1a3ced1b'
Target       : [ 704643072 1640242675 1069511448 3444212592 1326423982 1616727891
  476011096  468532250]
Out Hash     : [ 184549376 1862567351  535652379 2274727596  363921549 3056832580
 1017252847  881494100]
Nonce   : 751417039
Hash    : b'0000000bb785046f1b68ed1fac9695878d00b115449033b6ef0ba23c54888a34'
Out Hash: b'0000000bb785046f1b68ed1fac9695878d00b115449033b6ef0ba23c54888a34'
